In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
trajectories_train_df = pd.read_csv('../data/original/trajectories(table 5)_training.csv',parse_dates=['starting_time'])
trajectories_test_df = pd.read_csv('../data/test/original/trajectories(table 5)_test1.csv',parse_dates=['starting_time'])
trajectories_df = pd.concat([trajectories_train_df, trajectories_test_df])
trajectories_df = trajectories_df.reset_index(drop=True)

In [3]:
link_df = pd.read_csv('../data/original/links (table 3).csv', index_col=0)

In [4]:
def parseSingleTravelSeq(i, s):
    nodes = []
    seq = s['travel_seq'].split(';');
    
    for k, n in enumerate(seq):
        
        info = n.split('#')
        link_id = int(info[0])
        start_time = datetime.datetime.strptime(info[1], '%Y-%m-%d %H:%M:%S')
        consume = float(info[2])
    
        end_time = datetime.datetime.strptime(info[1], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(seconds=consume)
        node = {}
        node['start_time'] = start_time
        node['end_time'] = end_time
        node['link'] = link_id
        node['consume'] = consume
        node['index'] = i
        node['length'] = link_df.loc[link_id]['length']
        nodes.append(node)
        
    return nodes

In [5]:
single_nodes = []
for i in trajectories_df.index.values:
    if i%2000 == 0:
        print(i)
    single_nodes.extend(parseSingleTravelSeq(i, trajectories_df.loc[i]))

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000
82000
84000
86000
88000
90000
92000
94000
96000
98000
100000
102000
104000
106000
108000
110000


In [6]:
single_travel_link_df = pd.DataFrame(single_nodes)

In [7]:
twenty_timedelta = datetime.timedelta(minutes=20)
forty_timedelta = datetime.timedelta(minutes=40)
def normlize_start_time(start_time):
    minute = start_time.minute
    second = start_time.second
    
    current_timedelta = datetime.timedelta(minutes=minute, seconds=second)
    
    if current_timedelta < twenty_timedelta:
        return start_time - current_timedelta
    elif current_timedelta < forty_timedelta:
        return start_time - current_timedelta + twenty_timedelta
    else:
        return start_time - current_timedelta + forty_timedelta

In [8]:
def normlize_hour_time(start_time):
    return start_time.hour

In [9]:
def normlize_date(start_time):
    return start_time.date()

In [11]:
single_travel_link_df['date'] = single_travel_link_df['start_time'].apply(normlize_date)
single_travel_link_df['twenty_minutes_date'] = single_travel_link_df['start_time'].apply(normlize_start_time)
single_travel_link_df['hour_time'] = single_travel_link_df['start_time'].apply(normlize_hour_time)

# 排除每天的异常值

In [13]:
groupby = single_travel_link_df.groupby(['link', 'date'])

Q_TOP_LIMIT = groupby['consume'].quantile(0.95)
Q_TOP_LIMIT = Q_TOP_LIMIT.reset_index()

top_pivot_df = pd.DataFrame.pivot_table(Q_TOP_LIMIT, values='consume', index=['link'], columns=['date'])

def DiscreteFilter(s):
    
    if (s.name % 10000) == 0:
        print(s.name)
    
    top_consume = top_pivot_df[s['date']][s['link']]

    if s['consume'] >= top_consume:
        return 'Top'
    else:
        return 'Normal'
    
single_travel_link_df['state'] = single_travel_link_df.apply(DiscreteFilter, axis=1)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000


In [14]:
single_travel_link_df[single_travel_link_df['state'] == 'Top']

,consume,end_time,index,length,link,start_time,date,twenty_minutes_date,hour_time,state
9,104.79,2016-07-19 00:38:24.790,1,197,122,2016-07-19 00:36:40,2016-07-19,2016-07-19 00:20:00,0,Top
30,17.87,2016-07-19 00:58:16.870,4,84,101,2016-07-19 00:57:59,2016-07-19,2016-07-19 00:40:00,0,Top
54,14.65,2016-07-19 01:36:47.650,7,40,108,2016-07-19 01:36:33,2016-07-19,2016-07-19 01:20:00,1,Top
76,82.87,2016-07-19 01:40:02.870,9,144,113,2016-07-19 01:38:40,2016-07-19,2016-07-19 01:20:00,1,Top
119,36.31,2016-07-19 01:49:25.310,15,59,123,2016-07-19 01:48:49,2016-07-19,2016-07-19 01:40:00,1,Top
124,34.41,2016-07-19 01:49:21.410,16,78,105,2016-07-19 01:48:47,2016-07-19,2016-07-19 01:40:00,1,Top
138,26.15,2016-07-19 02:11:33.150,18,23,103,2016-07-19 02:11:07,2016-07-19,2016-07-19 02:00:00,2,Top
181,151.71,2016-07-19 02:46:51.710,25,121,111,2016-07-19 02:44:20,2016-07-19,2016-07-19 02:40:00,2,Top
240,45.33,2016-07-19 03:55:27.330,35,23,103,2016-07-19 03:54:42,2016-07-19,2016-07-19 03:40:00,3,Top
284,11.21,2016-07-19 04:27:12.210,43,40,108,2016-07-19 04:27:01,2016-07-19,2016-07-19 04:20:00,4,Top


In [15]:
valid_travel_link_df = single_travel_link_df[(single_travel_link_df['state'] == 'Normal')]

In [16]:
valid_travel_link_df.drop(['state'], axis=1).to_csv('../data/preprocess/valid_travel_link.csv', index=False)